# Build `barcode_runs.csv` file
This notebook takes the file labeled `{date}_barcode_runs.csv` for each day that an Illumina sequencing run was done, and maps it to the files that contain the Illumina R1 reads from that day.
Sometimes samples are sequenced on multiple days. So next, we concatenate the files for each sample, separated by a semicolon.

The final data frame should have only one row for each `sample` x `library`. 

In [1]:
import glob
import itertools
import numpy
import pandas as pd
from IPython.display import display, HTML

List of Illumina sequencing runs and their subdirectory. 

The file names are like this:

`expt_0_lib1_trans_S1_R1_001.fastq.gz`

which is a concatenation, separated by "_" of the columns:
* `experiment`
* `library`
* `sort_bin`

`sample` is a concatenation, separated by "-" of the columns:
* `experiment`
* `antibody`
* `concentration`
* `sort_bin`

Sometimes samples get mixed up during the whole sorting and/or library prep process.
This is easiest to detect (and fix) when lib1 and lib2 are simply swapped.

I am basically going to hardcode this in when this happens. 

In [2]:
runs = {
    '210504':'/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney',
}

In [3]:
barcode_runs = pd.DataFrame()

for run in runs:
    print(run)
    df = (
        pd.read_csv(f'{run}_barcode_runs.csv')
        .assign(
            sample=lambda x: x['experiment']+'-'+x['antibody']+'-'+x['concentration']+'-'+x['sort_bin'],
            R1=lambda x: runs[run]+'/'+x['experiment']+'_'+x['library']+'_'+x['sort_bin']+'_*R1*.fastq.gz',
            )
        )
    
    assert len(df.groupby(['library', 'sample'])) == len(df)
    barcode_runs = pd.concat([barcode_runs, df], ignore_index=True)
    
R1_df=(barcode_runs[['library', 'sample', 'R1']])
R1_df=R1_df.groupby(['library', 'sample'])['R1'].apply(lambda x: '; '.join(x)).reset_index()

barcode_runs = (barcode_runs
                .drop(columns=['R1'])
                .drop_duplicates()
                .merge(R1_df,
                       how='left',
                       on=['library', 'sample'],
                       validate='one_to_one',
                      )
               )

assert len(barcode_runs.groupby(['library', 'sample'])) == len(barcode_runs)
barcode_runs.to_csv('../barcode_runs.csv', index=False)

display(HTML(barcode_runs.head().to_html()))

210504


,date,experiment,library,antibody,concentration,sort_bin,sample,experiment_type,number_cells,frac_escape,R1
0,210419,expt_0,lib1,none,0,trans,expt_0-none-0-trans,transformation,NaN,NaN,/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney/expt_0_lib1_trans_*R1*.fastq.gz
1,210419,expt_0,lib2,none,0,trans,expt_0-none-0-trans,transformation,NaN,NaN,/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney/expt_0_lib2_trans_*R1*.fastq.gz
2,210419,expt_1,lib1,RBD_expr_sort,sat,RBD,expt_1-RBD_expr_sort-sat-RBD,RBD_expr_sort,17612661.0,0.570,/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney/expt_1_lib1_RBD_*R1*.fastq.gz
3,210419,expt_1,lib2,RBD_expr_sort,sat,RBD,expt_1-RBD_expr_sort-sat-RBD,RBD_expr_sort,17597458.0,0.570,/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney/expt_1_lib2_RBD_*R1*.fastq.gz
4,210419,expt_2,lib1,SortSeq,sat,bin1,expt_2-SortSeq-sat-bin1,SortSeq,1243000.0,0.444,/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney/expt_2_lib1_bin1_*R1*.fastq.gz


Test that this will not break the `snakemake` pipeline.

In [4]:
barcode_runs_expandR1 = (
    barcode_runs
    .assign(R1=lambda x: x['R1'].str.split('; ').map(
                    lambda y: list(itertools.chain(*map(glob.glob, y)))),
            n_R1=lambda x: x['R1'].map(len),
            sample_lib=lambda x: x['sample'] + '_' + x['library'],
            )
    )

assert barcode_runs_expandR1['sample_lib'].nunique() == len(barcode_runs_expandR1)
if any(barcode_runs_expandR1['n_R1'] < 1):
    raise ValueError(f"no R1 for {barcode_runs_expandR1.query('n_R1 < 1')}")
    
display(HTML(barcode_runs_expandR1.head().to_html()))

,date,experiment,library,antibody,concentration,sort_bin,sample,experiment_type,number_cells,frac_escape,R1,n_R1,sample_lib
0,210419,expt_0,lib1,none,0,trans,expt_0-none-0-trans,transformation,NaN,NaN,[/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney/expt_0_lib1_trans_S1_R1_001.fastq.gz],1,expt_0-none-0-trans_lib1
1,210419,expt_0,lib2,none,0,trans,expt_0-none-0-trans,transformation,NaN,NaN,[/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney/expt_0_lib2_trans_S2_R1_001.fastq.gz],1,expt_0-none-0-trans_lib2
2,210419,expt_1,lib1,RBD_expr_sort,sat,RBD,expt_1-RBD_expr_sort-sat-RBD,RBD_expr_sort,17612661.0,0.570,[/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney/expt_1_lib1_RBD_S3_R1_001.fastq.gz],1,expt_1-RBD_expr_sort-sat-RBD_lib1
3,210419,expt_1,lib2,RBD_expr_sort,sat,RBD,expt_1-RBD_expr_sort-sat-RBD,RBD_expr_sort,17597458.0,0.570,[/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney/expt_1_lib2_RBD_S4_R1_001.fastq.gz],1,expt_1-RBD_expr_sort-sat-RBD_lib2
4,210419,expt_2,lib1,SortSeq,sat,bin1,expt_2-SortSeq-sat-bin1,SortSeq,1243000.0,0.444,[/shared/ngs/illumina/agreaney/210504_D00300_1221_AHK3JFBCX3/Unaligned/Project_agreaney/expt_2_lib1_bin1_S5_R1_001.fastq.gz],1,expt_2-SortSeq-sat-bin1_lib1
